<a href="https://colab.research.google.com/github/hsieh2000/HW/blob/main/historic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#匯入套件
import datetime
from datetime import timedelta
import time
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import os
import random
import pymysql
import warnings

config_path = "../../setting.json"                          #設定setting.json路徑
with open(config_path) as f:
    config = json.load(f)                                   #以json形式讀取setting.json
    root = config["root"]                                   #取得root路徑

# 建立資料夾
newest_path = f'{root}最新檔/'                              #設定最新檔匯出路徑
log_path = f'{root}歷史淨值記錄檔/'                         #設定記錄檔輸出路徑
dir = ["csv/", "txt/", "err/"]                              #設定資料夾名稱

for i in dir:
  if not os.path.isdir(log_path+i):                         #判斷是否資料夾存在，如果不存在則建立
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):                          #判斷是否資料夾存在，如果不存在則建立
    os.mkdir(newest_path)

# 路徑設定
basicInfo_path = f'{newest_path}境外基金基本資訊.csv'
bug_fund = "歷史淨值錯誤基金代碼"
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]
c = pd.read_csv(f'{basicInfo_path}')                        #讀取最新基本資訊檔

# 參數設定
delay = random.uniform(1, 5)                                #設定隨機數1至5
warnings.filterwarnings("ignore")                           #忽略警告訊息
useragent = UserAgent()                                     #調用 UserAgent() 使用者代理
header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "Referer": "https://announce.fundclear.com.tw",   
    "user-agent" : useragent.random                         #設定requests header
}

class Net(object):
    def __init__(self):
        a = self.date_list()
        a = self.crawler(a)
        self.store(a[0],a[1])
        self.to_mysql(a[0])

    def get_last_month_start_end(self):                                                     #取得上月第一天與最後一天
        today = datetime.date.today()
        last_day = datetime.date(today.year, today.month, 1) - timedelta(1)
        first_day = datetime.date(last_day.year, last_day.month, 1)
        return (first_day, last_day)

    def date_list(self):                                                                    #生成上月全部天數
        date = self.get_last_month_start_end()[0]
        last_day = self.get_last_month_start_end()[1]
        last_month = []
        while last_day >= date:
            last_month.append(date.strftime("%Y/%m/%d"))
            date+=timedelta(days=1)
                    
        return last_month

    def Retry(self, url, payload):                                                          #當爬蟲失敗時會自動嘗試重新爬取，最多三次                                                                                                                         
        times = 0
        while times<3 :
            try:
                session = requests.Session()
                res = session.post(url, headers = header, data = payload, timeout=30)
                soup = BeautifulSoup(res.text, 'html.parser')
                return soup
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass

    def crawler(self, dates):                                                               #爬取資料
        with open(f'{err_path}{bug_fund}.txt', mode='w') as D:
            D.write("Error ID:\n")
            D.close()
        df = pd.DataFrame(columns = ["日期","基金代碼","淨值"])
        S = dates[0].split('/')                                                             #將日期中的"/"去除，用於檔名
        E = dates[-1].split('/')
        filename = f"歷史淨值{''.join(S)}~{''.join(E)}"
        with open(f'{txt_path}{filename}.txt', mode='w') as f:
            f.write('{\"data\":[')

        for j,i in enumerate(c["基金代碼"]):
            try:
                payload = {                                                                 #設定post請求的payload參數
                    "agentId": "",
                    "fundId": i,
                    "days": "",
                    "beginDate": dates[0],
                    "endDate": dates[-1],
                    "checkSubmit": "Y"
                }
                with open(f'{txt_path}{filename}.txt', mode='a') as f:
                    f.write('{\"基金代碼\":')
                    f.write(f'\"{i}\",\"細項\":')

                url = 'https://announce.fundclear.com.tw/MOPSFundWeb/D02_02.jsp'

                soup = self.Retry(url, payload)

                raw = str(soup.select('script')[-1])                                        #取得網頁資料
                date = re.compile("\d{4}/\d{2}/\d{2}")                                      #定義日期表達式
                net = re.compile("\d+\.\d{6}")                                              #定義淨值表達式
                date = re.findall(date,raw)                                                 #取得符合日期表達式元素
                net = re.findall(net,raw)                                                   #取得符合淨值表達式元素
                Jlist = list(map(lambda x,y : {                                             #將資料儲存為list of json形式
                    "日期":x,
                    "淨值":y
                },date,net ))
                with open(f'{txt_path}{filename}.txt', mode='a') as f:                      #將資料寫入txt
                    f.write(str(Jlist).replace("\'","\""))
                    f.write("}")
                    if j+1 == len(c["基金代碼"]):
                      pass
                    else:
                      f.write(",")

                Jlist_df = list(map(lambda x,y : {                                          #將資料儲存為list of json形式(給df用)
                    "日期":x,
                    "基金代碼":i,
                    "淨值":y
                },date,net ))
                df = df.append(Jlist_df, ignore_index=True)                                 #將資料新增至Dataframe
                print(f'{i},{j}/{len(c["基金代碼"])}')
                time.sleep(delay)
            except:
                with open(f'{err_path}{bug_fund}.txt', mode='a') as D:                      #爬取失敗則寫入錯誤代碼至txt檔
                            D.write(f'{i},')
                            D.write("\n")
                            D.close() 
                print(f"{i} got problem")#這裡要改url=>i
                pass    
            #測試 
            # if j == 10:
            #     break

        with open(f'{txt_path}{filename}.txt', mode='a') as f:
            f.write(']}')

        df['日期'] =  pd.to_datetime(df['日期'], format="%Y-%m-%d")                         #將日期由str轉換為datetime                           
        return (df,filename)

    def store(self, df, filename):
        newest_filename = filename.split("2")[0]                                            #匯出到最新檔的去除日期
        print(f'export {filename}.csv')#把這行一道最後一行
        df.to_csv(f'{csv_path}{filename}.csv', encoding='utf-8')
        df.to_csv(f'{newest_path}{newest_filename}.csv', encoding='utf-8')

    def to_mysql(self, df):                                                                 #匯入資料庫
        df=df.fillna(0)
        db_settings = {
            "host": "163.14.137.58",
            "port": 7733,
            "user": "ftca",
            "password": "ftca",
            "db": "raw_data",
            "charset": "utf8"
        }
        # 建立Cursor物件
        conn = pymysql.connect(**db_settings)

        with conn.cursor() as cursor:
            # 新增資料SQL語法
            command = "INSERT INTO history_net_amtc(`DATE`, FUND_ID, NAV)VALUES(%s, %s, %s)"
            for i in range(len(df)):
                cursor.execute(
                    command, (df.iloc[i][0], df.iloc[i][1], df.iloc[i][2]))
                print(f"第{i}筆成功")
            # 儲存變更

            cursor.execute("UPDATE history_net_amtc SET NAV = NULL WHERE NAV = 0")
            print("saved data to mysql!")
            conn.commit()
        
if __name__ == "__main__":
    n = Net()